# Step 3: Tool 실행하기 (Agentic Loop)

이 노트북에서는 AI가 호출한 도구를 **실제로 실행**하고 결과를 전달하는 방법을 배웁니다.

**학습 목표:**
- 도구 실행 함수 작성
- AI에게 결과 전달 (tool_result)
- Agentic Loop 완성

---

## 1. Agentic Loop란?

AI가 도구를 호출하면, 우리가 실행하고 결과를 돌려주는 **반복 과정**입니다.

```
사용자 질문: "지금 몇 시야?"
        ↓
AI: "get_current_time 호출할게요" (stop_reason="tool_use")
        ↓
우리: 도구 실행 → "현재 시간: 14:30"
        ↓
AI에게 결과 전달 (tool_result)
        ↓
AI: "현재 시간은 14시 30분입니다." (stop_reason="end_turn")
```

---

## 2. 패키지 설치 및 모델 선택

In [1]:
!pip install anthropic -q
print("설치 완료!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 12.2 MB/s eta 0:00:00
설치 완료!


In [2]:
# ===== Claude 4.5 모델 선택 =====
# 아래 세 가지 모델 중 하나를 선택하세요 (주석 해제)

# 1. Haiku 4.5 - 가장 빠르고 저렴 (테스트용 추천)
# MODEL = "claude-haiku-4-5-20251001"

# 2. Sonnet 4.5 - 균형 잡힌 성능
MODEL = "claude-sonnet-4-5-20250929"

# 3. Opus 4.5 - 최고 성능 (비용 높음)
# MODEL = "claude-opus-4-5-20251101"

print(f"선택된 모델: {MODEL}")

선택된 모델: claude-sonnet-4-5-20250929


---

## 3. API 키 설정

In [3]:
import os
from datetime import datetime
from anthropic import Anthropic

# API 키 설정
# os.environ["ANTHROPIC_API_KEY"] = "sk-ant-여기에-API-키-입력"

try:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
except:
    pass

client = Anthropic()
print("준비 완료!")

준비 완료!


---

## 4. Tool 정의

In [4]:
# 현재 시간 조회 도구
GET_TIME_TOOL = {
    "name": "get_current_time",
    "description": "현재 날짜와 시간을 조회합니다.",
    "input_schema": {
        "type": "object",
        "properties": {}
    }
}

TOOLS = [GET_TIME_TOOL]
print(f"등록된 도구: {[t['name'] for t in TOOLS]}")

등록된 도구: ['get_current_time']


---

## 5. 도구 실행 함수 작성

AI가 도구를 호출하면 **실제로 실행**할 함수가 필요합니다.

In [5]:
def execute_tool(tool_name, tool_input):
    """
    AI가 요청한 도구를 실행합니다.

    Parameters:
        tool_name: 도구 이름
        tool_input: 도구 입력 파라미터

    Returns:
        str: 실행 결과
    """
    if tool_name == "get_current_time":
        now = datetime.now()
        return f"현재: {now.strftime('%Y년 %m월 %d일 %H시 %M분')}"

    return f"알 수 없는 도구: {tool_name}"

# 테스트
result = execute_tool("get_current_time", {})
print(f"도구 실행 결과: {result}")

도구 실행 결과: 현재: 2025년 12월 04일 07시 26분


---

## 6. AI의 Tool 호출 확인

In [6]:
# AI에게 시간을 물어봄
response = client.messages.create(
    model=MODEL,
    max_tokens=200,
    tools=TOOLS,
    messages=[{"role": "user", "content": "지금 몇 시야?"}]
)

print(f"stop_reason: {response.stop_reason}")

# Tool 호출 정보 확인
if response.stop_reason == "tool_use":
    for block in response.content:
        if block.type == "tool_use":
            print(f"\n도구 호출 감지!")
            print(f"  이름: {block.name}")
            print(f"  ID: {block.id}")
            print(f"  입력: {block.input}")

stop_reason: tool_use

도구 호출 감지!
  이름: get_current_time
  ID: toolu_01KPHQyKcPPcseo9G6CsuW86
  입력: {}


---

## 7. Agentic Loop 구현

이제 모든 것을 연결합니다!

In [ ]:
def chat_with_tool(user_message):
    """
    Tool을 사용하는 챗봇.

    Parameters:
        user_message: 사용자 메시지

    Returns:
        str: AI의 최종 응답
    """
    messages = [{"role": "user", "content": user_message}]

    # 최대 3번 반복 (무한 루프 방지)
    for i in range(3):
        print(f"\n--- 반복 {i+1} ---")

        # API 호출
        response = client.messages.create(
            model=MODEL,
            max_tokens=500,
            tools=TOOLS,
            messages=messages
        )

        print(f"stop_reason: {response.stop_reason}")

        # Case 1: AI가 도구를 호출하려는 경우
        if response.stop_reason == "tool_use":
            # 도구 블록 찾기
            for block in response.content:
                if block.type == "tool_use":
                    print(f"도구 호출: {block.name}")

                    # 도구 실행
                    result = execute_tool(block.name, block.input)
                    print(f"실행 결과: {result}")

                    # 메시지에 AI 응답 추가
                    messages.append({
                        "role": "assistant",
                        "content": response.content
                    })

                    # 메시지에 도구 결과 추가 (중요!)
                    messages.append({
                        "role": "user",
                        "content": [{
                            "type": "tool_result",
                            "tool_use_id": block.id,  # 어떤 도구 호출에 대한 결과인지
                            "content": result         # 도구 실행 결과
                        }]
                    })

                    break  # 하나의 도구만 처리

            continue  # 다음 반복

        # Case 2: 최종 응답 (도구 호출 없음)
        for block in response.content:
            if block.type == "text":
                return block.text

    return "최대 반복 횟수 초과"


# 테스트
print("=" * 50)
print("질문: 지금 몇 시야?")
print("=" * 50)

answer = chat_with_tool("지금 몇 시야?")

print("\n" + "=" * 50)
print(f"최종 답변: {answer}")
print("=" * 50)

질문: 지금 몇 시야?

--- 반복 1 ---
stop_reason: tool_use
도구 호출: get_current_time
실행 결과: 현재: 2025년 12월 03일 12시 28분

--- 반복 2 ---
stop_reason: end_turn

최종 답변: 현재 시간은 **2025년 12월 3일 12시 28분**입니다.


---

## 8. 핵심 코드 설명

### tool_result 전달 형식

```python
{
    "role": "user",
    "content": [{
        "type": "tool_result",
        "tool_use_id": block.id,  # 어떤 도구 호출에 대한 결과?
        "content": result         # 실행 결과 문자열
    }]
}
```

- `tool_use_id`: 반드시 AI가 호출한 도구의 ID와 일치해야 함
- `content`: 도구 실행 결과 (문자열)

---

## 9. 계산기 도구 추가해보기

In [ ]:
# 계산기 도구 추가
CALCULATOR_TOOL = {
    "name": "calculator",
    "description": "두 숫자의 사칙연산을 수행합니다.",
    "input_schema": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "첫 번째 숫자"},
            "b": {"type": "number", "description": "두 번째 숫자"},
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "연산 종류"
            }
        },
        "required": ["a", "b", "operation"]
    }
}

# 도구 목록 업데이트
TOOLS = [GET_TIME_TOOL, CALCULATOR_TOOL]

# 실행 함수 업데이트
def execute_tool(tool_name, tool_input):
    if tool_name == "get_current_time":
        now = datetime.now()
        return f"현재: {now.strftime('%Y년 %m월 %d일 %H시 %M분')}"

    elif tool_name == "calculator":
        a = tool_input["a"]
        b = tool_input["b"]
        op = tool_input["operation"]

        if op == "add":
            result = a + b
        elif op == "subtract":
            result = a - b
        elif op == "multiply":
            result = a * b
        elif op == "divide":
            result = a / b if b != 0 else "오류: 0으로 나눌 수 없음"

        return f"{a} {op} {b} = {result}"

    return f"알 수 없는 도구: {tool_name}"

print(f"등록된 도구: {[t['name'] for t in TOOLS]}")

등록된 도구: ['get_current_time', 'calculator']


In [ ]:
# 계산 테스트
print("질문: 123 곱하기 456은?")
print("-" * 40)

answer = chat_with_tool("123 곱하기 456은?")

print("\n" + "=" * 50)
print(f"최종 답변: {answer}")

질문: 123 곱하기 456은?
----------------------------------------

--- 반복 1 ---
stop_reason: tool_use
도구 호출: calculator
실행 결과: 123 multiply 456 = 56088

--- 반복 2 ---
stop_reason: end_turn

최종 답변: 123 곱하기 456은 **56,088**입니다.


---

## 10. 정리

이번 단계에서 배운 내용:

| 개념 | 설명 |
|------|------|
| Agentic Loop | 도구 호출 → 실행 → 결과 전달 → 반복 |
| `stop_reason="tool_use"` | AI가 도구 호출을 원함 |
| `tool_use_id` | 도구 호출의 고유 ID |
| `tool_result` | 도구 실행 결과를 AI에게 전달 |
| 반복 제한 | 무한 루프 방지를 위해 최대 횟수 설정 |

### Agentic Loop 흐름

```
사용자 질문
    ↓
API 호출 (tools 등록)
    ↓
stop_reason 확인
    ├── "tool_use" → 도구 실행 → 결과 전달 → 다시 API 호출
    └── "end_turn" → 최종 응답 반환
```

### 다음 단계

다음 노트북에서는 **다중 Tool**을 배웁니다.
- 여러 도구를 동시에 등록
- AI가 상황에 맞는 도구 자동 선택